In [7]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
import plotly.io as pio

SEAS5 = pd.read_csv('../../data/UNSEEN/heatwave-2012/SEAS5_events.csv')
ERA5 = pd.read_csv('../../data/UNSEEN/heatwave-2012/ERA5_events.csv')
SEAS5_clean = SEAS5.dropna(subset=['t2m', 'year', 'tprate'])
ERA5_clean = ERA5.dropna(subset=['t2m', 'year', 'tp'])


In [11]:
fig = px.scatter(SEAS5_clean, x='t2m', y='tprate', trendline='ols',
                 labels={'t2m': 'Temperature (t2m)', 'tprate': 'Precipitation Rate (tprate)'},
                 title='Relationship between Temperature and Precipitation Rate')

fig.show()

In [9]:
# Add an interaction term between 't2m' and 'year'
SEAS5_clean['t2m_year_interaction'] = SEAS5_clean['t2m'] * SEAS5_clean['year']

# Prepare the data for regression analysis
X = SEAS5_clean[['t2m', 'year', 't2m_year_interaction']]
X = sm.add_constant(X)
y = SEAS5_clean['tprate']

# Fit the regression model
model = sm.OLS(y, X).fit()
print(model.summary())

## this shows there is a statistically significant relationship between an interaction term on year and temperature on precipitation rate. 

                            OLS Regression Results                            
Dep. Variable:                 tprate   R-squared:                       0.612
Model:                            OLS   Adj. R-squared:                  0.611
Method:                 Least Squares   F-statistic:                     1977.
Date:                Fri, 05 Apr 2024   Prob (F-statistic):               0.00
Time:                        09:55:43   Log-Likelihood:                 25491.
No. Observations:                3771   AIC:                        -5.097e+04
Df Residuals:                    3767   BIC:                        -5.095e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.7021 

/var/folders/c_/5rx_wyrs0pg78x3j1_zxrpl00000gn/T/ipykernel_14491/3662294129.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [19]:
fig = go.Figure()

SEAS5_clean['decade'] = (SEAS5['year'] // 10) * 10

for decade in SEAS5_clean['decade'].unique():
    subset = SEAS5_clean[SEAS5_clean['decade'] == decade]
    fig.add_trace(go.Scatter(x=subset['t2m'], y=subset['tprate'],
                             mode='markers', name=f'Decade {decade}'))

fig.update_layout(
    title='SEAS5 Temperature vs. Precipitation Rate Over Decades',
    xaxis_title='Temperature (K)', 
    yaxis_title='Precipitation Rate (tprate)',
    width=900,  # Set the width of the plot
    height=500  # Set the height of the plot
)

fig.show()

/var/folders/c_/5rx_wyrs0pg78x3j1_zxrpl00000gn/T/ipykernel_14491/3826791852.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
# Save the figure as a PNG file
pio.write_image(fig, '../../figures/UNSEEN/temperature_vs_precipitation.png')

# Save the figure as an HTML file
pio.write_html(fig, '../../figures/UNSEEN/temperature_vs_precipitation.html')

In [17]:
for SEAS5_clean in SEAS5_clean['decade'].unique():
    subset = SEAS5_clean[SEAS5_clean['decade'] == decade]
    correlation = subset[['t2m', 'tprate']].corr().iloc[0, 1]
    print(f"Decade: {decade}, Correlation: {correlation}")

Decade: 1980, Correlation: -0.7308954412856354
Decade: 1990, Correlation: -0.7449318403061287
Decade: 2000, Correlation: -0.782076057304732
Decade: 2010, Correlation: -0.7696317747118551
Decade: 2020, Correlation: -0.8277109557074999


In [20]:
# Save the figure as a PNG file
pio.write_image(fig, 'drought/v3/figures/precipitation_vs_temperature.png')

# Save the figure as an HTML file
pio.write_html(fig, 'drought/v3/figures/precipitation_vs_temperature.html')

In [3]:
return_periods_df = pd.read_csv('../Data/drought/v3/temp_return_periods_for_plotting.csv')
return_periods_df = return_periods_df[return_periods_df['return_period'] <= 100]
return_periods_df.head()

,return_period,model_1,model_2
0,1.01,291.186343,293.499712
1,1.11,291.725359,294.097652
2,1.21,291.941793,294.337747
3,1.31,292.093707,294.506268
4,1.41,292.213972,294.639679


In [3]:
import plotly.graph_objects as go

# Create a trace for model_1
trace1 = go.Scatter(
    x=return_periods_df['return_period'],
    y=return_periods_df['model_1'],
    mode='lines',
    name='20 years ago'
)

# Create a trace for model_2
trace2 = go.Scatter(
    x=return_periods_df['return_period'],
    y=return_periods_df['model_2'],
    mode='lines',
    name='Today'
)

# Define the layout
layout = go.Layout(
    title='Risk of Crop Failure Temperatures have Jumped Up',
    xaxis=dict(title='Return Period (years)'),
    yaxis=dict(title='Temperature (K)'),
)

# Create a Figure and add the traces
return_period_fig = go.Figure(data=[trace1, trace2], layout=layout)

# Add a horizontal line at y = 296.061
return_period_fig.add_shape(
    type="line",
    x0=return_periods_df['return_period'].min(),
    y0=296.061,
    x1=return_periods_df['return_period'].max(),
    y1=296.061,
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
)

return_period_fig.add_annotation(
    x=90,
    y=295.7,
    text="2012 Drought Temperature",
    showarrow=False,
    font=dict(
        size=12,
        color="Black"
    ),
)

# Show the figure
return_period_fig.show()

NameError: name 'return_periods_df' is not defined

In [44]:
# Save the figure as a PNG file
pio.write_image(return_period_fig, 'drought/v3/figures/temp_return_period.png')

# Save the figure as an HTML file
pio.write_html(return_period_fig, 'drought/v3/figures/temp_return_period.html')

In [4]:
return_periods_df = pd.read_csv('../Data/drought/v3/temp_return_periods_for_plotting.csv')
return_periods_df = return_periods_df[return_periods_df['return_period'] <= 100]
return_periods_df['model_1'] = return_periods_df['model_1'] - 273.15
return_periods_df['model_2'] = return_periods_df['model_2'] - 273.15

return_periods_df.head()

,return_period,model_1,model_2
0,1.01,18.036343,20.349712
1,1.11,18.575359,20.947652
2,1.21,18.791793,21.187747
3,1.31,18.943708,21.356268
4,1.41,19.063972,21.489679


In [12]:
# Define the layout
layout = go.Layout(
    title='Risk of Crop Failure Temperatures have Jumped<br><sub>Return periods for extreme temperatures in the US breadbasket</sub>',
    xaxis=dict(
        title='Return Period (years)',
        range=[return_periods_df['return_period'].min(), return_periods_df['return_period'].max()]  # Set the range of the x-axis
    ),
    yaxis=dict(
        title='3-month Mean Temperature (°C)',
        showticklabels=True, 
        range=[return_periods_df['model_1'].min(), return_periods_df['model_2'].max()]  # Set the range of the y-axis
    ),
)

# Create a trace for model_1
trace1 = go.Scatter(
    x=return_periods_df['return_period'],
    y=return_periods_df['model_1'],
    mode='lines',
    line=dict(color='rgba(0,0,255,0.5)'),
    showlegend=False,  # Hide the legend
    name="1982-1985",
    hovertemplate='Return Period: %{x}<br>Temperature: %{y:.2f}'
)

# Create a trace for model_2
trace2 = go.Scatter(
    x=return_periods_df['return_period'],
    y=return_periods_df['model_2'],
    mode='lines',
    line=dict(color='rgba(255,0,0,0.5)'),
    fill='tonexty',
    showlegend=False,  # Hide the legend
    name = "2020-2023",
    hovertemplate='Return Period: %{x}<br>Temperature: %{y:.2f}'
)

# Create a Figure and add the traces
return_period_fig_simple = go.Figure(data=[trace1, trace2], layout=layout)

# Add a label for the "20 years ago" line
return_period_fig_simple.add_annotation(
    x=return_periods_df['return_period'].max(),  # Move the annotation to the right of the plot
    y=return_periods_df['model_1'].iloc[-1],
    text="1982-1985",
    showarrow=False,
    font=dict(
        size=12,
        color="rgba(0,0,255,0.5)"
    ),
    xshift=37  # Shift the annotation to the right
)

# Add a label for the "Today" line
return_period_fig_simple.add_annotation(
    x=return_periods_df['return_period'].max(), # Move the annotation to the right of the plot
    y=return_periods_df['model_2'].iloc[-1],
    text="2020-2023",
    showarrow=False,
    font=dict(
        size=12,
        color="rgba(255,0,0,0.5)"
    ),
    xshift=37  # Shift the annotation to the right
)

return_period_fig_simple.add_annotation(
    x=return_periods_df['return_period'].max(),
    #y=(return_periods_df['model_1'].iloc[-1] + return_periods_df['model_2'].iloc[-1]) / 2,  # Position the arrow in the middle of the two lines
    y = 25,
    text="",
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=0,
    ay=60,
)

# Add a label for the arrow
return_period_fig_simple.add_annotation(
    x=return_periods_df['return_period'].max(),  # Move the annotation to the right of the plot
    y=24.1,  # Move the annotation up
    text="Increased<br>Risk",
    showarrow=False,
    font=dict(
        size=12,
        color="#636363"
    ),
    xshift=35
)

# Add a horizontal line at y = 296.061
return_period_fig_simple.add_shape(
    type="line",
    x0=return_periods_df['return_period'].min(),
    y0=22.911,
    x1=return_periods_df['return_period'].max(),
    y1=22.911,
    line=dict(
        color="Gray",
        width=2,
        dash = "dashdot"
    ),
)

return_period_fig_simple.add_annotation(
    x=50,
    y=23.2,
    text="2012 Heatwave",
    showarrow=False,
    font=dict(
        size=12,
        color="Gray"
    ),
)

# return_period_fig_simple.add_shape(
#     type="line",
#     x0=5,
#     y0=return_periods_df['model_1'].min(),
#     x1=5,
#     y1=return_periods_df['model_2'].max(),
#     line=dict(
#         color="Gray",
#         width=2
#     ),
# )

# # Add a new annotation below the plot area
# return_period_fig_simple.add_annotation(
#     x=5,  # Position the annotation at x = 5
#     y=return_periods_df['model_1'].min(),  # Position the annotation below the plot area
#     text="The return period for that heatwave today is every 5 years",
#     showarrow=True,
#     ax=120,
#     ay=40
# )

# Show the figure
return_period_fig_simple.show()

In [13]:
# Save the figure as a PNG file
pio.write_image(return_period_fig_simple, 'drought/v3/figures/temp_return_period.png')

# Save the figure as an HTML file
pio.write_html(return_period_fig_simple, 'drought/v3/figures/temp_return_period.html')